In [6]:
# --------------------
# Simulation Notebook
# --------------------
from google.colab import drive
drive.mount('/content/drive')

# Cambiar al proyecto
import os, sys
project_path = '/content/drive/MyDrive/Proyecto-IA-DeteccionDeFraudeEnTransacciones'
%cd $project_path

# --------------------
# Añadir src y todas sus subcarpetas al path
# --------------------
src_path = os.path.join(project_path, 'src')
for root, dirs, files in os.walk(src_path):
    if root not in sys.path:
        sys.path.append(root)

# --------------------
# Importar módulos
# --------------------
import pandas as pd
import joblib

from scoring.simulator import simulate_and_score as original_simulate_and_score # Import generate_transaction
from scoring.realtime import score_transaction
from features.feature_engineering import add_features

# --------------------
# Directorios
# --------------------
models_dir = "./models/trained_models/"
data_processed = "./data/processed/"

# --------------------
# Cargar modelos entrenados
# --------------------
rf_model = joblib.load(os.path.join(models_dir, "rf_model.pkl"))
xgb_model = joblib.load(os.path.join(models_dir, "xgb_model.pkl"))

# --------------------
# Cargar datos limpios
# --------------------
df = pd.read_csv(os.path.join(data_processed, "creditcard_clean.csv"))
df = add_features(df)

# --------------------
# Simulación y scoring
# --------------------
# Modify generate_transaction to use the same columns as df (excluding 'Class')
def generate_transaction_with_features(df):
    # For demonstration, let's just take a random row from df and remove 'Class'
    # In a real scenario, you would generate a new transaction based on the data distribution
    transaction = df.sample(1).drop('Class', axis=1)
    return transaction

# Modify simulate_and_score to accept a generate_transaction_func argument
def simulate_and_score(model, generate_transaction_func, num_simulations=1000):
    simulated_transactions = []
    fraud_probabilities = []

    for _ in range(num_simulations):
        transaction = generate_transaction_func()
        simulated_transactions.append(transaction)
        # Assuming score_transaction can handle a single transaction (DataFrame row)
        # or convert it to the expected format
        prob = score_transaction(transaction, model)
        fraud_probabilities.append(prob)

    # Convert the list of transaction DataFrames to a single DataFrame
    if simulated_transactions:
        simulated_transactions_df = pd.concat(simulated_transactions, ignore_index=True)
    else:
        simulated_transactions_df = pd.DataFrame() # Return an empty DataFrame if no simulations

    return simulated_transactions_df, fraud_probabilities


print("Simulación con RandomForest:")
trans_rf, prob_rf = simulate_and_score(rf_model, generate_transaction_func=lambda: generate_transaction_with_features(df)) # Pass the modified function

print("\nSimulación con XGBoost:")
trans_xgb, prob_xgb = simulate_and_score(xgb_model, generate_transaction_func=lambda: generate_transaction_with_features(df)) # Pass the modified function

# Primeras 5 transacciones simuladas RF
print("\nPrimeras 5 transacciones simuladas RF:")
print(trans_rf.head())
print("Probabilidades de fraude:", prob_rf[:5])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Proyecto-IA-DeteccionDeFraudeEnTransacciones
Simulación con RandomForest:

Simulación con XGBoost:

Primeras 5 transacciones simuladas RF:
       Time        V1        V2        V3        V4        V5        V6  \
0  143361.0 -0.909455  1.817140 -1.658070  0.153591  0.601791 -1.752530   
1  152799.0 -0.568846 -1.001052  2.875814 -2.860101 -1.314317  1.566861   
2   45721.0  0.921037 -1.247312 -0.294450 -0.512318 -0.722317 -0.004112   
3   79455.0 -3.091999 -1.686752 -0.089100 -0.107105 -3.218276  1.424510   
4   56581.0 -0.522966  0.966812  1.567872  0.730408  0.085781 -0.056854   

         V7        V8        V9  ...       V21       V22       V23       V24  \
0  0.346396  0.533494 -0.491068  ...  0.333957  0.785924 -0.241700 -0.229745   
1 -1.450378  0.605342 -0.750438  ... -0.094106  0.539791 -0.607618 -1.051516   
2 -0.127986 -0.040